In [ ]:
%config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style, quantity_support
plt.style.use(astropy_mpl_style)
quantity_support()

rc('figure',figsize=(16,8))
rc('font',size=12)


import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_moon, get_sun

In [ ]:
Salta = EarthLocation(lat=-24.731358*u.deg, lon=-65.409535*u.deg, height=1152*u.m)
utcoffset = -3*u.hour  # Eastern Daylight Time

door_az = 120.
az_span = 50.
el_min = 30.
el_max = 50.


In [ ]:


####### Dates for June 2022
# day0 = 18
# ndays = 7
# dates = ['2022-6-{} 00:00:00'.format(day0+i) for i in range(ndays)]

####### Dates for July 2022
day0 = 19
ndays = 7
dates = ['2023-4-{} 00:00:00'.format(day0+i) for i in range(ndays)]

delta_time = np.linspace(utcoffset.value, 24+utcoffset.value, 1000)*u.hour

fill_between([door_az-az_span/2, door_az+az_span/2], [el_max,el_max], y2=[el_min, el_min], color='g', alpha=0.3, label='Observing Window' )
fill_between([door_az-az_span/2-15, door_az+az_span/2-15], [el_max,el_max], y2=[el_min, el_min], color='b', alpha=0.3, label='Observing Window (shift mount by 15 deg)' )


for d in dates:
    ### Time setting
    time0 = Time(d)-utcoffset
    alltimes = time0 + delta_time
    
    ### Local coordinates
    frame_Salta = AltAz(obstime=alltimes, location=Salta)
    
    ### Moon
    moon_Salta = get_moon(alltimes)
    moonaltazs_Salta = moon_Salta.transform_to(frame_Salta)  
    
    ### Plot
    plot(moonaltazs_Salta.az, moonaltazs_Salta.alt, 'o', label=d + ' (Local)')
    #print(moonaltazs_Salta.az, moonaltazs_Salta.alt)

legend()
#xlim((120-45)*u.deg, (120+45)*u.deg)
ylim(0*u.deg, 90*u.deg)
xlabel('Azimuth')
ylabel('Elevation')
title('Moon Visibility from Salta from {} (local)'.format(dates[0]))


# Now for a specific day in local time

In [ ]:
date = '2023-5-16 00:00:00'
delta_time = np.linspace(0, 24, 1000)*u.hour

fill_between([door_az-az_span/2, door_az+az_span/2], [el_max,el_max], y2=[el_min, el_min], color='g', alpha=0.3, label='Observing Window' )
fill_between([door_az-az_span/2-15, door_az+az_span/2-15], [el_max,el_max], y2=[el_min, el_min], color='b', alpha=0.3, label='Observing Window (shift mount by 15 deg)' )

time0 = Time(date)-utcoffset
alltimes = time0 + delta_time

### Local coordinates
frame_Salta = AltAz(obstime=alltimes, location=Salta)

### Moon
moon_Salta = get_moon(alltimes)
moonaltazs_Salta = moon_Salta.transform_to(frame_Salta)  

### Moon
sun_Salta = get_sun(alltimes)
sunaltazs_Salta = sun_Salta.transform_to(frame_Salta)  

valid = (moonaltazs_Salta.alt.value < (el_max+1)) & (moonaltazs_Salta.alt.value > (el_min-1)) & (moonaltazs_Salta.az.value > 0) & (moonaltazs_Salta.az.value < 360)

scatter(moonaltazs_Salta.az[valid], moonaltazs_Salta.alt[valid],
            c=delta_time[valid], label='Moon', lw=0, s=200, marker='o', cmap='jet')

plot(sunaltazs_Salta.az[valid], sunaltazs_Salta.alt[valid], 'r--', lw=3, label='Sun')

colorbar().set_label('Local Time')
legend()
xlim((-180)*u.deg, (360)*u.deg)
ylim(25*u.deg, 55*u.deg)
xlabel('Azimuth')
ylabel('Elevation')
title('Moon Visibility from Salta {} (local)'.format(date))
